Author: Shailendra Chalasani
Intention: Supervised Learning Likes to dislikes ratio - Linear Regression

In [17]:
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import pandas as pd
from collections import Counter, defaultdict
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
df_video_dislikes = pd.read_csv(f"../../data/cleaned_datasets/df_video_dislikes.csv")
print(len(df_video_dislikes))
df_video_dislikes.dropna(inplace=True)
print(len(df_video_dislikes))

37422
37264


In [19]:
df_video_dislikes.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'title', 'published_at',
       'view_count', 'likes_count', 'dislikes_count', 'comment_count', 'tags',
       'description', 'comments', 'topic1', 'topic2', 'topic3'],
      dtype='object')

In [20]:
# df_video_dislikes = df_video_dislikes[['title','view_count','likes_count','dislikes_count','comment_count','tags','description','comments']]
df_video_dislikes = df_video_dislikes[['view_count','likes_count','dislikes_count','comment_count','topic1','topic2','topic3']]
# df_video_dislikes['likes_dislikes_ratio'] = df_video_dislikes['likes_count']/df_video_dislikes['dislikes_count']
df_video_dislikes

,view_count,likes_count,dislikes_count,comment_count,topic1,topic2,topic3
0,1048888,19515,226,1319,2,9,3
1,15352638,359277,7479,18729,5,7,1
2,925281,11212,401,831,1,2,9
3,2641597,39131,441,3745,5,3,7
4,715724,32887,367,1067,5,0,1
...,...,...,...,...,...,...,...
37417,120408275,2180780,35871,81360,5,0,8
37418,2841917,20759,1049,2624,6,9,7
37419,13346678,720854,4426,90616,1,7,8
37420,252057,19198,1234,1471,9,8,0


In [21]:
from sklearn.preprocessing import StandardScaler

X = df_video_dislikes[['view_count','likes_count','comment_count','topic1','topic2','topic3']]
y =df_video_dislikes['dislikes_count']

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

scaler = StandardScaler().fit(train_X)
X_train = scaler.transform(train_X)
X_test = scaler.transform(test_X)

In [22]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np

# Create the LinearRegression object
linear_reg = LinearRegression()

# Define the parameter grid for grid search
param_grid = {
    'fit_intercept': [True, False]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=linear_reg, param_grid=param_grid, scoring='r2', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(train_X, train_y)

# Get the mean and standard deviation of the R2 scores
r2_scores = grid_search.cv_results_['mean_test_score']
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)

# Get the best parameters
best_params = grid_search.best_params_

print("Mean R2 Score:", r2_mean)
print("Standard Deviation of R2 Score:", r2_std)
print("Best Parameters:", best_params)

# If the R-squared (R2) score is negative, it means that the model performs worse than a horizontal line fitted to the data.
# In other words, the model's predictions are not capturing the underlying patterns or variability in the data, and it might be a poor fit for the given problem.

Mean R2 Score: 0.5545743841367532
Standard Deviation of R2 Score: 8.226168792052624e-05
Best Parameters: {'fit_intercept': True}


In [28]:
# MSE is used to evaluate the performance of estimators. A lower MSE indicates that the estimator is more accurate. MSE can also be used to compare the performance of different estimators.
from sklearn.inspection import permutation_importance

# Train the DecisionTreeRegressor model
lreg = LinearRegression()
lreg.fit(train_X, train_y)

# Calculate permutation importances
result = permutation_importance(lreg, test_X, test_y, n_repeats=10)

# Get the feature importances and their indices
importances = result.importances_mean
indices = np.argsort(importances)

# Print the feature importances
for f in range(train_X.shape[1]):
    print(f"{train_X.columns[indices[f]]}: {importances[indices[f]]}")

topic2: -3.279076882533039e-05
topic1: -3.6161815099133852e-06
comment_count: 9.96339445512473e-05
topic3: 0.00026474972196400827
view_count: 0.15187198936017507
likes_count: 0.1653049994890819


In [25]:
from sklearn.dummy import DummyRegressor

dummy_reg_mean = DummyRegressor(strategy="mean")
dummy_reg_median = DummyRegressor(strategy="median")

dummy_reg_mean.fit(train_X,train_y)
dummy_reg_median.fit(train_X,train_y)

mean_predicted= dummy_reg_mean.predict(test_X)
median_predicted=dummy_reg_median.predict(test_X)

# dtree_r2 = r2_score(test_y, predicted_dtree)
mean_r2 = r2_score(test_y, mean_predicted)
median_r2 = r2_score(test_y, median_predicted)

print("lreg_r2=", r2_mean)
print("dummy mean_r2=", mean_r2)
print("dummy median_r2=", median_r2)

lreg_r2= 0.5545743841367532
dummy mean_r2= -7.723054153152553e-05
dummy median_r2= -0.01570734921459449


In [27]:
pd.options.display.float_format = '{:.2f}'.format
test_X.describe()[['likes_count', 'view_count']]

,likes_count,view_count
count,9316.00,9316.00
mean,166349.75,5615131.83
std,489320.25,20604303.85
min,97.00,24022.00
25%,13361.50,502971.75
50%,42380.50,1322318.50
75%,131037.75,3642234.75
max,13641573.00,518378576.00


In [ ]:
#In statistics, the coefficient of determination, or R-squared, is a measure of how well a model fits a set of data. It is calculated as the ratio of the variance explained by the model to the total variance in the data.
r_squared = r2_score(test_y, predicted)
r_squared

# This means that the linear regression model explains 67.24% of the variance in the data.

In [ ]:
from sklearn.metrics import mean_absolute_error

#If the MAE is 0, this means that the predicted values are perfectly accurate.
mae = mean_absolute_error(test_y, predicted)
mae

In [ ]:
df = pd.DataFrame()
df['likes_views_ratio']= test_y
df['Predicted']=predicted
df['Error']=df['likes_views_ratio']-df['Predicted']
df

Higher mutual information scores indicate a stronger dependency or relevance between the feature and the target variable.
You can interpret the scores to identify the most informative features for your KNNRegressor model.

In [14]:
from sklearn.feature_selection import mutual_info_regression

# Calculate the mutual information for each feature
mi_scores = mutual_info_regression(train_X, train_y)

# Print the mutual information scores
for feature, score in zip(train_X.columns, mi_scores):
    print(f"{feature}: {score}")

view_count: 0.007675722535267493
likes_count: 0.008597343431237636
comment_count: 0.0


Positive correlation values indicate a positive relationship between the feature and the target variable,
while negative correlation values indicate a negative relationship.
Higher absolute correlation values indicate a stronger correlation between the feature and the target variable.

In [15]:
# Concatenate the feature columns and target variable into a single dataframe
data = pd.concat([train_X, pd.Series(train_y, name='Target')], axis=1)

# Calculate the correlation matrix
correlation_matrix = data.corr()

# Get the correlation values between features and the target variable
target_correlation = correlation_matrix['Target'].drop('Target')

# Print the correlation values
for feature, correlation in target_correlation.items():
    print(f"{feature}: {correlation}")

view_count: -0.07569412880687132
likes_count: 0.10658741039746743
comment_count: 0.01519041093722882
